In [11]:
'''COPY PASTED CORE ON 05/02/2025 FROM ROUND-BENEDETTO'''
'''UPDATE AS NEEDED'''
import numpy as np
from numpy import random as npr
from scipy.special import comb
import itertools
cooperate = np.array( [ 1, 0 ] )
defect = np.array( [ 0, 1 ] )
# defining all strategies
# # template function
# parameter_list = [ k, ]
# def strategy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
#     # move either cooperate or defect
#     move
#     return move 

# STRATEGIES:________________________________________________
    # template function:
    # def strategy( round_number = 0, match_history = [[]], player_index = 0 ) : 
    #     # move either cooperate or defect
    #     move = cooperate/defect
    #     return move 

# nice guy function: always cooperate
def nice_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :      # 'None' al posto di [[]] ?
    move = cooperate
    return move 

# bad guy function: always defect 
def bad_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :        # 'None' al posto di [[]] ?
    move = defect
    return move 

# mainly nice guy: randomly defect k% of the times and cooperate 100-k %, k<50
def mainly_nice( parameter_list = [ 0.4 ], round_number = 0, match_history = [[]], player_index = 0 ) :# eccezione da gestire meglio?
    k = parameter_list[ 0 ]
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < k ) : move = defect
    else : move = cooperate
    return move

# mainly bad guy: randomly defect k% of the times and cooperate 100-k %, k>50
def mainly_bad( parameter_list = [ 0.6 ], round_number = 0, match_history = [[]], player_index = 0 ) :                                                                           # eccezione da gestire meglio?
    k = parameter_list[0]
    u = npr.random()                                                                            # corretto in random() invece di random(1)
    if ( u < 1-k ) : move = defect
    else : move = cooperate
    return move

# random guy: randomly defect or cooperate
def random_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :                                                                            # eccezione da gestire meglio?
    u = npr.random()                                                                           # corretto in random() invece di random(1)
    if ( u < 0.5 ) : move = defect
    else : move = cooperate
    return move

# tit_for_tat function: start by cooperating, then repeat what the opponent has done in the previous move
def tit_for_tat( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = cooperate
    else : move = match_history[ round_number - 1 ][ 1 - player_index ]
    return move

# resentful guy: cooperate until the opponent defects once, then always defect
def resentful_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    move = cooperate
    if ( round_number > 0 ) :
        for res in match_history :
            if np.all( res[ 1 - player_index  ] == defect ) :
                move = defect
    return move

# trusting guy (increasing odds of cooperating according to match history):
# initially is like random guy (coop/defect at k=50%), then updates k
def trusting_guy( parameter_list = [ 0.5 ], round_number = 0, match_history = [[]], player_index = 0 ) :
    k = parameter_list[0]    # initial trust
    # environmental trust
    if ( round_number > 0 ) :
        coop = 0.         # numero di cooperazioni avversarie finora
        # 'res[ 1 - player_index  ][ 0 ]' prende il primo elemento della mossa avversaria (1 se ha cooperato, 0 se ha defectato)
        for res in match_history : coop += res[ 1 - player_index  ][ 0 ]
        k = coop / len( match_history )           # updated k
    u = npr.random()                                                                          # corretto in random() invece di random(1)
    if ( u > k ) : move = defect
    else : move = cooperate
    return move

# balancing guy (cooperates only for an even cooperation history, otherwise defect)
def balancing_guy( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) :
    move = defect
    if ( round_number > 0 ) :
        # total cooperation seen
        coop = 0
        for res in match_history : coop += ( res[ 0 ][ 0 ] + res[ 1 ][ 0 ])
        if ( ( coop % 2 ) == 0 ) : move = cooperate
    return move

# Mid resentful: cooperates on the first move, and defects if the opponent has defected on any of the previous 3 moves, else cooperates
def mid_resentful( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0):
    move = cooperate
    # checko gli ultimi 3 turni (o meno se sono all'inizio):
    start = max(0, round_number - 3)
    for round_i in range(start, round_number):
        # controllo se la mossa avversaria corrisponde ad un defect=[0,,1]
        if np.array_equal(match_history[round_i][1 - player_index], defect):   
            move = defect
            break  # non serve continuare il controllo, esco dal for
    return move

# reverse tit_for_tat function: start by defecting, then does the opposite of what the opponent has done in the previous move
def reverse_tit_for_tat( parameter_list = [], round_number = 0, match_history = [[]], player_index = 0 ) : 
    if ( round_number == 0 ) : move = defect
    else : move = [1,1] - match_history[ round_number - 1 ][ 1 - player_index ]  
            # in this way i obtain the opposite move: [1,1] - [1,0] = [0,1] and [1,1] - [0,1] = [1,0]
    return move

# dictionary of all strategies
strategies = {
    'NiceGuy' : nice_guy,
    'BadGuy' : bad_guy,
    'MainlyNice' : mainly_nice,
    'MainlyBad' : mainly_bad,
    'RandomGuy': random_guy,
    'TitForTat' : tit_for_tat,
    'ResentfulGuy' : resentful_guy,
    'TrustingGuy' : trusting_guy,
    'Thanos' : balancing_guy,
    'MidResentful': mid_resentful,
    'ReverseTft': reverse_tit_for_tat,
}

# default Payoff matrix
default_R = 2
default_S = 0
default_T = 3
default_P = 1
Payoff = np.array( [ [ default_R , default_S ], [ default_T, default_P ] ] )

# function to play one round 1 vs 1 for two different* strategies players
def round( key_1, key_2, parameter_list_1, parameter_list_2, round_number, match_history, M = Payoff ) :
    # getting strategies from dictionary
    S_1 = strategies[ key_1 ]
    S_2 = strategies[ key_2 ]
    # computing next move for each player
    u_1 = S_1( parameter_list_1, round_number, match_history, 0 )
    u_2 = S_2( parameter_list_2, round_number, match_history, 1 )
    # computing rewards for each player
    r_1 = np.dot( u_1, Payoff.dot(u_2))
    r_2 = np.dot( u_2, Payoff.dot(u_1))
    # updating match history
    match_history.append( [ u_1, u_2 ] )
    # returning results
    return r_1, r_2

# function to play a match of N rounds 1 vs 1 for two different* strategies player
def match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
    # match history strarting empty
    history = [ ]
    # starting total rewards
    R_1 = 0
    R_2 = 0
    # list of partial sums
    Rewards = []
    for round_i in range( 0, N_rounds ) :
        r_1, r_2 = round( key_1, key_2, parameter_list_1, parameter_list_2, round_i, history, M )
        R_1 += r_1
        R_2 += r_2
        Rewards.append( [ R_1, R_2 ] )
    return Rewards
    


In [8]:
def tourney(participants={},N_gironi=1,N_rounds = 10, M = Payoff):
    # Retrieve only the names of the strategies repeated as many times as their frequency allows.
    player_list_from_dict = []
    for item in (list(participants.keys())):
        for i in range (participants[item][0]):
            player_list_from_dict.append(item)
    # Compute the combinations
    b = list(itertools.combinations(player_list_from_dict, 2))
    N_giornata = len(b)
    # List which will be used to have the scores of each match by the side of the two opponents
    list_with_scores = b
    # Loop over the combinations one by one
    for i in range (N_giornata):
        # Player1 and Player2 strategies
        player1 = b[i][0]
        player2 = b[i][1]
        # Player1 and Player2 parameters: [0] is the frequency, [1] is the list of parameters
        param1  = participants[player1][1]
        param2  = participants[player2][1]
        # Initiliaze an empty list to store the results over the match
        AllRewards = []
        # Do the match between the two players and update the rewards; The ordered syntax for match is:
        # match( key_1, key_2, parameter_list_1, parameter_list_2, N_rounds = 10, M = Payoff ) : 
        AllRewards =  match(player1, player2, param1, param2, N_rounds, M)
        # Retrieve the final scores for Player1 and Player2 as the last element in the list
        R1, R2 = AllRewards[-1][0], AllRewards[-1][1]
        list_with_scores[i] += (R1,R2)
    # Un-comment the following print to debug
    #for item in list_with_scores: print(item)
    
    # Good, now each combination has the corresponding score: we now need to assign it to the respective players and calculate the final score
    strat = list(participants.keys())
    freq = [participants[key][0] for key in strat]
    named =[]
    # Add identification to the layer trough the use of a numerical value from 1 up to their frequency BUT PRESERVE THE ORDER
    for key,f in zip (strat,freq):
        for j in range(1,f+1):
            if (f!=0):
                named.append(key + str(j))
    # Compute the permutations preserving the order
    with_names = list(itertools.combinations(named,2))
    # Copy the scores for each match over to the ones with identification
    # In list_with_scores[I][J] we have: I = ordered number of the combination, and J:
    # J=0 is 1st strategy; J=1 is 2nd strategy; J=2 is Player1 score; J=3 is Player2 score
    for i in range (N_giornata): with_names[i] += (list_with_scores[i][2],list_with_scores[i][3] )
    # Un-comment the following print to debug
    #for item in with_names: print(item)

    # Compute the score for each named player and add it to the list
    FinalScores = []
    for player in named:
        SumOver1 = np.sum([ with_names[i][2] for i in range (N_giornata) if (with_names[i][0] == player) ])
        SumOver2 = np.sum([ with_names[i][3] for i in range (N_giornata) if (with_names[i][1] == player) ])
        FinalScores.append(SumOver1 + SumOver2)
    Results = named
    
    for i in range (len(Results)):
        Results[i] = (Results[i],FinalScores[i])
    #Results += tuple( FinalScores )
    return ( Results )    

In [13]:
# Tourney test
test_part ={ 
    'NiceGuy' : (3,[]),
    'BadGuy' : (3,[]),
    'MainlyNice' : (4, [0.4] ),
    'MainlyBad' : (3, [0.6] ),
    'RandomGuy' : (1, []),
    'TitForTat' : (1,[]),
    'ResentfulGuy' : (2,[]),
    'TrustingGuy' : (1,[0.5]),
    'Thanos' : (2,[]),
    'MidResentful' : (1,[]),
    'ReverseTft': (2,[]),
}

result = tourney(test_part,N_gironi=1,N_rounds = 10, M = Payoff)
for item in result: print(item)
print("\nHere's the Ranking at the end of the tourney:")
# Sorted gives a copy, while using LIST.sort() does it in place.
Ranking = sorted(result,reverse=True, key=lambda x: x[1])
for item in Ranking: print(item)

('NiceGuy1', 262.0)
('NiceGuy2', 262)
('NiceGuy3', 260)
('BadGuy1', 430)
('BadGuy2', 434)
('BadGuy3', 428)
('MainlyNice1', 320)
('MainlyNice2', 297)
('MainlyNice3', 304)
('MainlyNice4', 301)
('MainlyBad1', 298)
('MainlyBad2', 312)
('MainlyBad3', 295)
('RandomGuy1', 312)
('TitForTat1', 334)
('ResentfulGuy1', 383)
('ResentfulGuy2', 382)
('TrustingGuy1', 327)
('Thanos1', 333)
('Thanos2', 333)
('MidResentful1', 375)
('ReverseTft1', 329)
('ReverseTft2', 339.0)

Here's the Ranking at the end of the tourney:
('BadGuy2', 434)
('BadGuy1', 430)
('BadGuy3', 428)
('ResentfulGuy1', 383)
('ResentfulGuy2', 382)
('MidResentful1', 375)
('ReverseTft2', 339.0)
('TitForTat1', 334)
('Thanos1', 333)
('Thanos2', 333)
('ReverseTft1', 329)
('TrustingGuy1', 327)
('MainlyNice1', 320)
('MainlyBad2', 312)
('RandomGuy1', 312)
('MainlyNice3', 304)
('MainlyNice4', 301)
('MainlyBad1', 298)
('MainlyNice2', 297)
('MainlyBad3', 295)
('NiceGuy1', 262.0)
('NiceGuy2', 262)
('NiceGuy3', 260)


In [12]:
print(result[0])

('NiceGuy1', 40.0)


In [13]:
print(result[0][0])

NiceGuy1


In [14]:
print(result[0][1])

40.0
